In [3]:
# required packages
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

# naviagete to get route name and route
driver = webdriver.Chrome()
driver.maximize_window()
driver.get("https://www.redbus.in/online-booking/west-bengal-transport-corporation?utm_source=rtchometile")
time.sleep(3)

scrolling = True
while scrolling:
    old_page = driver.page_source
    body = driver.find_element(By.TAG_NAME,'body')
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(4)
    new_page = driver.page_source
    if new_page == old_page:
        scrolling = False

page = driver.find_elements(By.CSS_SELECTOR,"a[class='route']")
route = [i.text for i in page]
link = [i.get_attribute('href') for i in page]
wbstc = pd.DataFrame({'route':route,'route_link':link})

wbstc_url = wbstc['route_link']

In [4]:
wbstc

,route,route_link
0,Digha to Kolkata,https://www.redbus.in/bus-tickets/digha-to-kol...
1,Kolkata to Digha,https://www.redbus.in/bus-tickets/kolkata-to-d...
2,Kolkata to Mandarmani,https://www.redbus.in/bus-tickets/kolkata-to-m...
3,Mandarmani to Kolkata,https://www.redbus.in/bus-tickets/mandarmani-t...


In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, WebDriverException
import time
import pandas as pd
import numpy as np

def initialize_driver():
    """Initialize the Chrome WebDriver."""
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def scroll_to_bottom(driver):
    """Scrolls to the bottom of the page to ensure all content is loaded."""
    last_height = driver.execute_script("return document.body.scrollHeight")
    scrolling = True
    while scrolling:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            scrolling = False
        else:
            last_height = new_height

def get_bus_data(urls):
    """Fetch bus data for each URL in the list."""
    all_bus_data = pd.DataFrame()
    driver = initialize_driver()

    for url in urls:
        try:
            driver.get(url)
            time.sleep(10)

            # Attempt to click the view buses button
            try:
                button = WebDriverWait(driver, 20).until(
                    EC.element_to_be_clickable((By.XPATH, '//div[@class="button"]'))
                )
                button.click()
            except TimeoutException:
                pass  # Continue if button not found or not clickable

            # Scroll to load all data
            scroll_to_bottom(driver)

            # Extract data
            bus_route = driver.find_element(By.CSS_SELECTOR, 'h1[class="D136_h1"]').text
            bus_name = [i.text for i in driver.find_elements(By.XPATH, '//div[@class="travels lh-24 f-bold d-color"]')]
            bus_type = [i.text for i in driver.find_elements(By.XPATH, '//div[@class="bus-type f-12 m-top-16 l-color evBus"]')]
            departure_time = [i.text for i in driver.find_elements(By.XPATH, '//div[@class="dp-time f-19 d-color f-bold"]')]
            duration = [i.text for i in driver.find_elements(By.XPATH, '//div[@class="dur l-color lh-24"]')]
            arrival_time = [i.text for i in driver.find_elements(By.XPATH, '//div[@class="bp-time f-19 d-color disp-Inline"]')]
            star_rating = [i.text for i in driver.find_elements(By.XPATH, '//div[@class="rating-sec lh-24"]')]
            price = [i.text for i in driver.find_elements(By.XPATH, '//span[@class="f-19 f-bold"]')]
            seats_available = [int(i.text.split()[0]) for i in driver.find_elements(By.XPATH, '//div[@class="seat-left m-top-30"]')]

            # Pad lists with NaN to match max length
            max_length = max(len(bus_name), len(bus_type), len(departure_time), len(duration),
                             len(arrival_time), len(star_rating), len(price), len(seats_available))

            def pad_list(lst, length):
                return lst + [np.nan] * (length - len(lst))

            bus_name = pad_list(bus_name, max_length)
            bus_type = pad_list(bus_type, max_length)
            departure_time = pad_list(departure_time, max_length)
            duration = pad_list(duration, max_length)
            arrival_time = pad_list(arrival_time, max_length)
            star_rating = pad_list(star_rating, max_length)
            price = pad_list(price, max_length)
            seats_available = pad_list(seats_available, max_length)

            # Create DataFrame for current URL
            wbstc_bus = pd.DataFrame({
                'Route Name': [bus_route] * max_length,
                'Route Link': [url] * max_length,
                'Bus Name': bus_name,
                'Bus Type': bus_type,
                'Departure Time': departure_time,
                'Duration': duration,
                'Arrival Time': arrival_time,
                'Star Rating': star_rating,
                'Price': price,
                'Seats Available': seats_available
            })

            # Append to main DataFrame
            all_bus_data = pd.concat([all_bus_data, wbstc_bus], ignore_index=True)

        except WebDriverException:
            print(f"Encountered an issue with URL: {url}. Moving to the next.")

    driver.quit()
    return all_bus_data

def process_bus_data(df):
    """Processes bus data by converting columns to numeric and filling NaNs."""
    df['Price'] = pd.to_numeric(df['Price'], errors='coerce')
    df['Star Rating'] = pd.to_numeric(df['Star Rating'], errors='coerce')
    df['Seats Available'] = pd.to_numeric(df['Seats Available'], errors='coerce')

    df['Price'] = df['Price'].fillna(df['Price'].mean().round(0).astype(int))
    df['Star Rating'] = df['Star Rating'].fillna(df['Star Rating'].mean().round(0).astype(int))
    df['Seats Available'] = df['Seats Available'].fillna(df['Seats Available'].mean().round(0).astype(int))
    
    return df

# Replace `bsrtc['route_link']` with the actual list of URLs.
urls = wbstc['route_link']
all_bus_data = get_bus_data(urls)
processed_bus_data = process_bus_data(all_bus_data)
processed_bus_data

,Route Name,Route Link,Bus Name,Bus Type,Departure Time,Duration,Arrival Time,Star Rating,Price,Seats Available
0,Digha to Kolkata Bus,https://www.redbus.in/bus-tickets/digha-to-kol...,SBSTC-DIGHA - GARIA - 09:25 (DIGHA DEPOT) - 3753,Non AC Seater (2+3),09:25,04h 30m,13:55,3.5,155.0,50.0
1,Digha to Kolkata Bus,https://www.redbus.in/bus-tickets/digha-to-kol...,SBSTC-DIGHA - HABRA - 10:00 (DIG DEPOT) - 3700,Non AC Seater (2+3),10:00,04h 30m,14:30,3.2,155.0,52.0
2,Digha to Kolkata Bus,https://www.redbus.in/bus-tickets/digha-to-kol...,SBSTC-DIGHA - BARASAT - 10:20 - 508,Non AC Seater (2+3),10:20,04h 30m,14:50,2.9,155.0,52.0
3,Digha to Kolkata Bus,https://www.redbus.in/bus-tickets/digha-to-kol...,SBSTC-DIGHA - GARIA - 10:30 (DIGHA DEPOT) - 2680,Non AC Seater (2+3),10:30,05h 30m,16:00,3.8,162.0,20.0
4,Digha to Kolkata Bus,https://www.redbus.in/bus-tickets/digha-to-kol...,SBSTC-DIGHA - BELGHARIA - 10:40 (BEL DEPOT) - ...,Non AC Seater (2+3),10:40,05h 00m,15:40,3.0,155.0,21.0
...,...,...,...,...,...,...,...,...,...,...
556,Mandarmani to Kolkata Bus,https://www.redbus.in/bus-tickets/mandarmani-t...,Subhadip Travels(Jagannath),A/C Executive (2+3),14:40,04h 15m,18:55,3.0,300.0,32.0
557,Mandarmani to Kolkata Bus,https://www.redbus.in/bus-tickets/mandarmani-t...,Aradhana Bus Service,A/C Seater / Sleeper (2+2),23:20,04h 40m,04:00,3.0,300.0,32.0
558,Mandarmani to Kolkata Bus,https://www.redbus.in/bus-tickets/mandarmani-t...,Ankita Paribahan,Non AC Seater (2+3),19:00,04h 00m,23:00,3.0,300.0,32.0
559,Mandarmani to Kolkata Bus,https://www.redbus.in/bus-tickets/mandarmani-t...,Aradhana Travels,Non AC Seater (2+3),22:15,05h 10m,03:25,3.0,300.0,32.0


In [6]:
processed_bus_data.isnull().sum()

Route Name         0
Route Link         0
Bus Name           0
Bus Type           0
Departure Time     0
Duration           0
Arrival Time       0
Star Rating        0
Price              0
Seats Available    0
dtype: int64

In [7]:
processed_bus_data.to_csv("wbstc.csv", index = False)